In [1]:
import pandas as pd
from borax.calendars.lunardate import LunarDate
import numpy as np
from boruta import BorutaPy


df = pd.read_csv('../data/train.csv')
df.columns = ['date','A','B']
df1 = pd.read_csv('../data/wkd_v1.csv')  #为EDA中的生成的节假日数据，其中0代表工作日，1为节假日
df1.columns = ['date','type']
df1.date = pd.to_datetime(df1.date)
df.date = pd.to_datetime(df.date)
df = df.merge(df1,on='date',how='left')

In [14]:
def yang2nong(date):
    
    year = int(date[:4])
    month = int(date[4:6])
    day = int(date[6:8])
    
    nong =  LunarDate.from_solar_date(year, month, day)
    
    res = nong.__format__('%y-%m-%d')
    return res

In [15]:
# 日期特征工程
def get_inner_date_feature_eng(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe["month"] = dataframe['date'].dt.month
    dataframe["year"] = dataframe['date'].dt.year
    
    dataframe["day"] = dataframe['date'].dt.day
    dataframe["day_of_week"] = dataframe['date'].dt.dayofweek
    dataframe["dayofy"] = dataframe['date'].dt.dayofyear
    dataframe["week"] = dataframe['date'].dt.week
    dataframe["quarter"] = dataframe['date'].dt.quarter

   

   
    
    dataframe['week_of_year'] = dataframe.date.dt.weekofyear


    dataframe['is_wknd'] = dataframe.date.dt.weekday // 4
    dataframe['is_month_start'] = dataframe.date.dt.is_month_start.astype(int)
    dataframe['is_month_end'] = dataframe.date.dt.is_month_end.astype(int)

    dataframe['week_block_num'] = [int(x) for x in np.floor((dataframe.date - pd.to_datetime('2017-12-31')).dt.days / 7) + 1]
    dataframe['quarter_block_num'] = (dataframe['year'] - 2018) * 4 + dataframe['quarter']
    dataframe['week_of_month'] = dataframe['week_of_year'].values // 4.35    
    #新增星期几时间变量
    dataframe['is_Mon'] = np.where(dataframe['day_of_week'] == 1, 1, 0)                                                                                       
    dataframe['is_Tue'] = np.where(dataframe['day_of_week'] == 2, 1, 0)                                                                                         
    dataframe['is_Wed'] = np.where(dataframe['day_of_week'] == 3, 1, 0)                                                                                         
    dataframe['is_Thu'] = np.where(dataframe['day_of_week'] == 4, 1, 0)                                                                                         
    dataframe['is_Fri'] = np.where(dataframe['day_of_week'] == 5, 1, 0)                                                                                         
    dataframe['is_Sat'] = np.where(dataframe['day_of_week'] == 6, 1, 0)                                                                                         
    dataframe['is_Sun'] = np.where(dataframe['day_of_week'] == 7, 1, 0)
    
    dataframe['day_of_month_10days']=np.where((dataframe['day']<=10) == 1, 1, 0)
    dataframe['day_of_month_20days']=np.where(((dataframe['day']>10)&(dataframe['day']<=20)) == 1, 1, 0)
    dataframe['day_of_month_30days']=np.where((dataframe['day']>20) == 1, 1, 0)
    dataframe['abs_month']=dataframe['day'].apply(lambda x: abs(x-16)+1)
    
    
    dataframe['CH_month'] = dataframe.China_date.dt.month  #农历月
    dataframe['CH_day_of_month'] = dataframe.China_date.dt.day #农历日
    dataframe['CH_day_of_year'] = dataframe.China_date.dt.dayofyear #农历的哪一天
    dataframe['abs_year']=dataframe['CH_day_of_year'].apply(lambda x: abs(x-182)+1)
    return dataframe


In [16]:
#进行农历转换
df['date1'] = df.date.dt.strftime('%Y%m%d')
df['China_date']=df['date1'].apply(lambda x: yang2nong(x))
#农历2月不同与公历2月，
#因此采用前向填充方式以构建pandas时间戳，难题在与20年存在的闰四月现象    
df.replace(['2018-2-29','2018-2-30','2019-2-29',
            '2020-2-29','2020-2-30'],np.nan,inplace=True)
    
df['China_date']=df['China_date'].fillna(method='ffill')
df['China_date'] = pd.to_datetime(df['China_date'])

In [17]:
k = df[df.date>='2019-06-01']
k1 = k[['date','A','type','China_date']]
k2 = k[['date','B','type','China_date']]

In [18]:
import numpy as np
k = k1.copy()
k = get_inner_date_feature_eng(k)
train1 = k[['type', 'month', 'year', 'day', 'day_of_week', 'dayofy',
       'week', 'quarter', 'week_of_year', 'is_wknd', 'is_month_start',
       'is_month_end', 'week_block_num', 'quarter_block_num', 'week_of_month',
       'is_Mon', 'is_Tue', 'is_Wed', 'is_Thu', 'is_Fri', 'is_Sat', 'is_Sun',
       'day_of_month_10days', 'day_of_month_20days', 'day_of_month_30days','abs_month', 'CH_month', 'CH_day_of_month',
       'CH_day_of_year','abs_year'

        ]][:-30]
train1_y = k['A'][:-30]

valid1 = k[['type', 'month', 'year', 'day', 'day_of_week', 'dayofy',
       'week', 'quarter', 'week_of_year', 'is_wknd', 'is_month_start',
       'is_month_end', 'week_block_num', 'quarter_block_num', 'week_of_month',
       'is_Mon', 'is_Tue', 'is_Wed', 'is_Thu', 'is_Fri', 'is_Sat', 'is_Sun',
       'day_of_month_10days', 'day_of_month_20days', 'day_of_month_30days','abs_month', 'CH_month', 'CH_day_of_month',
       'CH_day_of_year','abs_year'

           ]][-30:]
valid1_y = k['A'][-30:]

C:\Users\28947\Anaconda\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\28947\Anaconda\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [19]:
test_v1 = pd.read_csv('../data/test.csv')
test_v1.columns = ['date','A','B']
test_v1.date = pd.to_datetime(test_v1.date)
test_v1 = test_v1.merge(df1,on='date',how='left')


In [20]:

#进行农历转换
test_v1['date1'] = test_v1.date.dt.strftime('%Y%m%d')
test_v1['China_date']=test_v1['date1'].apply(lambda x: yang2nong(x))
#农历2月不同与公历2月，
#因此采用前向填充方式以构建pandas时间戳，难题在与20年存在的闰四月现象    
test_v1.replace(['2018-2-29','2018-2-30','2019-2-29',
            '2020-2-29','2020-2-30'],np.nan,inplace=True)
    
test_v1['China_date']=test_v1['China_date'].fillna(method='ffill')
test_v1['China_date'] = pd.to_datetime(test_v1['China_date'])
test_v1.columns

Index(['date', 'A', 'B', 'type', 'date1', 'China_date'], dtype='object')

In [21]:
test_v1_A = test_v1[['date','A','type','China_date']]
test_v1_A = get_inner_date_feature_eng(test_v1_A)
test_v1_A.columns

C:\Users\28947\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\28947\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\28947\Anaconda\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.i

Index(['date', 'A', 'type', 'China_date', 'month', 'year', 'day',
       'day_of_week', 'dayofy', 'week', 'quarter', 'week_of_year', 'is_wknd',
       'is_month_start', 'is_month_end', 'week_block_num', 'quarter_block_num',
       'week_of_month', 'is_Mon', 'is_Tue', 'is_Wed', 'is_Thu', 'is_Fri',
       'is_Sat', 'is_Sun', 'day_of_month_10days', 'day_of_month_20days',
       'day_of_month_30days', 'abs_month', 'CH_month', 'CH_day_of_month',
       'CH_day_of_year', 'abs_year'],
      dtype='object')

In [22]:
test = test_v1_A[['type', 'month', 'year', 'day', 'day_of_week', 'dayofy',
       'week', 'quarter', 'week_of_year', 'is_wknd', 'is_month_start',
       'is_month_end', 'week_block_num', 'quarter_block_num', 'week_of_month',
       'is_Mon', 'is_Tue', 'is_Wed', 'is_Thu', 'is_Fri', 'is_Sat', 'is_Sun',
       'day_of_month_10days', 'day_of_month_20days', 'day_of_month_30days','abs_month', 'CH_month', 'CH_day_of_month',
       'CH_day_of_year','abs_year']]


#cat_cols = ['type']
#train1[cat_cols] = train1[cat_cols].astype('category')
#valid1[cat_cols] = valid1[cat_cols].astype('category')
#test[cat_cols] = test[cat_cols].astype('category')

In [23]:
import pandas as pd
from tqdm import tqdm
import warnings
import lightgbm as lgb
import xgboost as xgb
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score,mean_squared_error
import re
import optuna
from optuna.samplers import TPESampler



warnings.filterwarnings('ignore')
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true))) 


In [24]:
train1.columns

Index(['type', 'month', 'year', 'day', 'day_of_week', 'dayofy', 'week',
       'quarter', 'week_of_year', 'is_wknd', 'is_month_start', 'is_month_end',
       'week_block_num', 'quarter_block_num', 'week_of_month', 'is_Mon',
       'is_Tue', 'is_Wed', 'is_Thu', 'is_Fri', 'is_Sat', 'is_Sun',
       'day_of_month_10days', 'day_of_month_20days', 'day_of_month_30days',
       'abs_month', 'CH_month', 'CH_day_of_month', 'CH_day_of_year',
       'abs_year'],
      dtype='object')

In [32]:
import lightgbm
train_matrix = lightgbm.Dataset(train1, train1_y)
test_matrix = lightgbm.Dataset(valid1, valid1_y)
evals = [(train_matrix, 'train'), (test_matrix, 'val')]

def my_mape(real_value, pre_value): 
    real_value, pre_value = np.array(real_value), np.array(pre_value)
    return np.mean(np.abs((real_value - pre_value) /( real_value+1)))
def eval_score(pre, train_set):
    real = train_set.get_label()
    score = my_mape(real, pre)
    return 'eval_score', score

In [34]:
train1

,type,month,year,day,day_of_week,dayofy,week,quarter,week_of_year,is_wknd,...,is_Sat,is_Sun,day_of_month_10days,day_of_month_20days,day_of_month_30days,abs_month,CH_month,CH_day_of_month,CH_day_of_year,abs_year
516,1,6,2019,1,5,152,22,2,22,1,...,0,0,1,0,0,16,4,28,118,65
517,1,6,2019,2,6,153,22,2,22,1,...,1,0,1,0,0,15,4,29,119,64
518,0,6,2019,3,0,154,23,2,23,0,...,0,0,1,0,0,14,5,1,121,62
519,0,6,2019,4,1,155,23,2,23,0,...,0,0,1,0,0,13,5,2,122,61
520,0,6,2019,5,2,156,23,2,23,0,...,0,0,1,0,0,12,5,3,123,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,0,9,2020,27,6,271,39,3,39,1,...,1,0,0,0,1,12,8,11,224,43
1001,0,9,2020,28,0,272,40,3,40,0,...,0,0,0,0,1,13,8,12,225,44
1002,0,9,2020,29,1,273,40,3,40,0,...,0,0,0,0,1,14,8,13,226,45
1003,0,9,2020,30,2,274,40,3,40,0,...,0,0,0,0,1,15,8,14,227,46


In [44]:
from sklearn.ensemble import RandomForestRegressor


from sklearn.ensemble import GradientBoostingRegressor
rf = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
     max_depth=5, random_state=2021, loss='ls')
boruta = BorutaPy(
   estimator = rf, 
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
)

boruta.fit(np.array(train1), np.array(train1_y))
# 输出结果
green_area = train1.columns[boruta.support_].to_list()
blue_area  = train1.columns[boruta.support_weak_].to_list()
print('features in the green area:', green_area)
print('features in the blue area:', blue_area)

features in the green area: ['day', 'dayofy', 'week', 'week_of_year', 'week_block_num', 'CH_day_of_month', 'CH_day_of_year', 'abs_year']
features in the blue area: ['abs_month']


In [36]:
len(green_area)

12

In [46]:
train1.columns[~boruta.support_]

Index(['type', 'month', 'year', 'day_of_week', 'quarter', 'is_wknd',
       'is_month_start', 'is_month_end', 'quarter_block_num', 'week_of_month',
       'is_Mon', 'is_Tue', 'is_Wed', 'is_Thu', 'is_Fri', 'is_Sat', 'is_Sun',
       'day_of_month_10days', 'day_of_month_20days', 'day_of_month_30days',
       'abs_month', 'CH_month'],
      dtype='object')

In [45]:
len(green_area)

8